# Uso do Spark data Sources

O **Spark Data Sources** é uma ferramenta poderosa no ecossistema Spark para carregar e salvar dados de várias fontes. Vamos dar uma olhada em como usá-lo!

Abaixo os principais tipos de sources:

| Tipo de Fonte de Dados | Descrição                                                                                     |
|------------------------|-----------------------------------------------------------------------------------------------|
| Parquet                | Formato de arquivo colunar eficiente e otimizado para análise que é amplamente suportado.    |
| ORC                    | Formato de arquivo colunar otimizado para consulta que é amplamente usado em ecossistemas do Hadoop. |
| JSON                   | Um formato de arquivo de dados textual amplamente utilizado.                                 |
| CSV                    | Formato de arquivo de texto separado por vírgulas.                                            |
| Avro                   | Estrutura de dados em formato binário compacto.                                                |
| Delta Lake             | Uma camada de armazenamento de dados do tipo "lakehouse" que oferece controle transacional, governança e otimizações. |
| Hadoop File System (HDFS) | Sistema de arquivos distribuído amplamente utilizado para armazenamento de dados.         |
| Cassandra              | Banco de dados distribuído NoSQL amplamente utilizado.                                       |
| Elasticsearch          | Motor de busca e análise em tempo real.                                                      |
| Kafka                  | Plataforma de streaming distribuída para processamento de eventos em tempo real.             |
| JDBC                   | Acesso a bancos de dados relacionais usando o Java Database Connectivity (JDBC).           |
| Table                 | Consulta de tabelas usando SQL.                                                                |

---
## DataFrames no Apache Spark

O Apache Spark é uma ferramenta poderosa para processamento de dados em larga escala e os DataFrames são uma parte fundamental dessa tecnologia. Um DataFrame é uma abstração de dados distribuídos que fornece uma interface fácil de usar para manipular dados em lote e em tempo real.

## Características Principais

Os DataFrames no Spark têm várias características essenciais:

- **Estrutura de Dados**: Um DataFrame é uma estrutura de dados tabular que se assemelha a uma tabela em um banco de dados relacional. Ele tem linhas e colunas, permitindo que você organize e manipule dados de maneira eficiente.

- **Tipagem de Colunas**: Cada coluna em um DataFrame possui um tipo de dados associado, permitindo que o Spark otimize o processamento e a execução de consultas.

- **Transformações e Ações**: Você pode realizar transformações complexas nos DataFrames, como filtragem, agregação e junção de dados. Além disso, você pode executar ações para coletar resultados ou gravar dados em fontes externas.

- **Integração com Diversas Fontes de Dados**: Os DataFrames no Spark podem ser usados com várias fontes de dados, como arquivos CSV, JSON, Parquet, bancos de dados relacionais, sistemas de mensagens, entre outros.

- **Suporte a SQL**: Você pode executar consultas SQL em DataFrames, tornando-o uma ferramenta poderosa para análise de dados.

## Carregando e Salvando Dados como dataframes

Documentação oficial:
- [Spark SQL Data Sources](https://spark.apache.org/docs/3.3.2/sql-data-sources.html)
- [Spark DataFrames](https://spark.apache.org/docs/3.3.2/api/python/reference/pyspark.sql/dataframe.html)

Aqui, vamos mostrar um exemplo de como carregar e salvar dados usando o Spark SQL Data Sources.

### Carregando Dados

Para carregar dados, você pode usar o método `read` do objeto `SparkSession`. Veja como:

```python
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Exemplo").getOrCreate()

# Carregando dados de um arquivo CSV
df = spark.read.csv("caminho/para/seu/arquivo.csv")
```

A tabela abaixo detalha algumas opções comuns que você pode passar para o método read:

| Opção            | Descrição                                    |
|------------------|----------------------------------------------|
| `format`         | Formato do arquivo (por ex., "csv", "parquet") |
| `option`         | Opções específicas do formato                |
| `header`         | Se a primeira linha contém nomes de colunas |
| `inferSchema`    | Inferir automaticamente o esquema dos dados  |


### Salvando Dados

Agora, vamos ver como salvar dados usando o método `write`:

```python
# Salvando o DataFrame em formato Parquet
df.write.parquet("caminho/para/seu/arquivo.parquet")
```

Aqui estão algumas opções que você pode usar ao salvar:

| Opção            | Descrição                                    |
|------------------|----------------------------------------------|
| `format`         | Formato de saída (por ex., "parquet", "csv") |
| `mode`           | Modo de escrita ("append", "overwrite", etc.) |
| `partitionBy`    | Colunas para particionar os dados           |
| `option`         | Opções específicas do formato                |

---
# Entendendo a leitura de dados

In [1]:
 from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Exemplo").config("spark.jars.packages", "org.postgresql:postgresql:42.2.24").getOrCreate()

In [ ]:
# Carregando dados de um arquivo CSV
df = (
    spark
    .read
    .csv("data/landing/VRA/")
)
df.first()

In [ ]:
df.show(1)

In [ ]:
df2 = (
    spark
    .read
    .format("csv")
    .option("inferSchema","true")
    .option("encoding","utf-8")
    .option("delimiter",";") #ou sep
    .load("data/landing/VRA/")
)

df2.printSchema()

In [ ]:
df3 = (
    spark
    .read
    .format("csv")
    .option("inferSchema","true")
    .option("encoding","utf-8")
    .option("delimiter",";") #ou sep
    .option("header", "true")
    .load("data/landing/VRA/")
)

df3.printSchema()

---
# Entendendo os schemas

In [ ]:
schema_json = df3.schema.json()
ddl = spark.sparkContext._jvm.org.apache.spark.sql.types.DataType.fromJson(schema_json).toDDL()

ddl

In [ ]:
from pyspark.sql.types import *

schema_ddl = """
    `ICAO Empresa Aerea` STRING,
    `Numero Voo` STRING,
    `Codigo DI` STRING,
    `Codigo Tipo Linha` STRING,
    `ICAO Aerodromo Origem` STRING,
    `ICAO Aerodromo Destino` STRING,
    `Partida Prevista` timestamp,
    `Partida Real` timestamp,
    `Chegada Prevista` timestamp,
    `Chegada Real` timestamp,
    `Situacao Voo` STRING,
    `Codigo Justificativa` STRING
"""

schema_method = StructType(
    [
        StructField('chegada_prevista', IntegerType()),
        StructField('chegada_real', IntegerType()),
        StructField('codigo_autorizacao', StringType()),
        StructField('codigo_justificativa', StringType()),
        StructField('codigo_tipo_linha', StringType()),
        StructField('icao_aerodromo_destino', StringType()),
        StructField('icao_aerodromo_origem', StringType()),
        StructField('icao_empresa_aerea', StringType()),
        StructField('numero_voo', IntegerType()),
        StructField('partida_prevista', IntegerType()),
        StructField('partida_real', IntegerType()),
        StructField('situacao_voo', StringType()),
    ]
)

In [ ]:
df4 = (
    spark
    .read
    .format("csv")
    .option("inferSchema","true")
    .option("encoding","utf-8")
    .option("delimiter",";") #ou sep
    .option("header", "true")
    .schema(schema_method)
    .load("data/landing/VRA/")
)

df3.printSchema()

In [ ]:
df4.count()

In [ ]:
df4.show(2, truncate=False, vertical=True)

---
# Entendendo o acesso a banco de dados com spark

In [9]:
# Leia dados do banco de dados usando o formato "jdbc"
df = (
    spark.read
    .format("jdbc")
    .option("driver", "org.postgresql.Driver")
    .option("url", "jdbc:postgresql://postgres:5432/data-way-poc?user=admin-dataway&password=IloveDataway") 
    .option("dbtable", "public.vehicle")
    .option("user", "admin-dataway")
    .option("password", "IloveDataway")
    .load()
)

# Exiba os dados lidos do banco de dados
df.show()

+---+-----------+--------------------+--------------------+-----------+-----------+----------------+--------------------+
| id|customer_id|          ano_modelo|              modelo| fabricante|ano_veiculo|       categoria|           dt_update|
+---+-----------+--------------------+--------------------+-----------+-----------+----------------+--------------------+
|  1|       1764|        2003 Audi S6|         Buick Regal|      smart|       2006|          Pickup|2023-11-03 12:54:...|
|  2|       6344|  2003 Suzuki Vitara|        BMW 5 Series|    Bentley|       2012|          Pickup|2023-11-03 12:54:...|
|  3|       9596|  2002 Toyota Avalon|GMC Sierra 1500 C...|     Subaru|       2010|     Van/Minivan|2023-11-03 12:54:...|
|  4|       3749|2009 Chevrolet Si...|       Tesla Model 3|  Chevrolet|       2010|             SUV|2023-11-03 12:54:...|
|  5|       8169|2000 Lincoln Cont...|Dodge Ram 3500 Re...|        BMW|       2009|     Van/Minivan|2023-11-03 12:54:...|
|  6|       5499|1992 Me

---
# Entendendo as escritas

In [ ]:
(
    df3
    .write
    .format("parquet")
    #.partitionBY()
    .mode("append")
    .save("/home/app/data/1.bronze")
)